### Carga y Normalización de Rutas y Mallas Geográficas en EPSG:4326 para Análisis Espacial Integrado

In [ ]:
import geopandas as gpd
import pandas as pd
import re

# Cargar las rutas desde el archivo SHP
gdf_rutas = gpd.read_file("archivo_atributos_unificados.shp")

# Cargar las mallas de la Península y de Canarias
gdf_malla_peninsula = gpd.read_file("../Ubicaciones y media/Mallas/Mallas_cuadrículas/Mallas Península/Malla10km_p.shp")
gdf_malla_canarias = gpd.read_file("../Ubicaciones y media/Mallas/Mallas_cuadrículas/Mallas Canarias/Malla10km_c.shp")

# Cargar el archivo CSV de las rutas
df_rutas_csv = pd.read_csv("archivo_rutas.csv")


In [3]:
# Asegurarse de que ambos GeoDataFrames usan el mismo CRS
gdf_rutas = gdf_rutas.to_crs("EPSG:4326")
gdf_malla_peninsula = gdf_malla_peninsula.to_crs("EPSG:4326")
gdf_malla_canarias = gdf_malla_canarias.to_crs("EPSG:4326")

In [4]:
gdf_rutas_canarias_test = gpd.sjoin(gdf_rutas, gdf_malla_canarias, how="left", predicate="intersects")

print(gdf_rutas_canarias_test['CUADRICULA'].head())

0    28RCS53
0    28RCS54
0    28RCS64
1    28RCS43
1    28RCS53
Name: CUADRICULA, dtype: object


### Asociación de Rutas a Cuadrículas de Mallas Peninsulares y Canarias y Fusión con Atributos CSV

In [5]:
# Unir las rutas con la malla de la Península
gdf_rutas_peninsula = gpd.sjoin(gdf_rutas, gdf_malla_peninsula, how="left", predicate="intersects")

# Unir las rutas con la malla de Canarias
gdf_rutas_canarias = gpd.sjoin(gdf_rutas, gdf_malla_canarias, how="left", predicate="intersects")

# Unir los resultados de las rutas en Península y Canarias
gdf_rutas_combinadas = pd.concat([gdf_rutas_peninsula, gdf_rutas_canarias], ignore_index=True)

gdf_rutas_combinadas = gdf_rutas_combinadas.groupby('ID_Ruta', as_index=False).agg({
    'CUADRICULA': lambda x: ', '.join(x.dropna().astype(str)),  # Mantiene todas las cuadriculas por ruta
    'geometry': 'first',  # Mantiene la geometría
    'source': 'first'  # Mantiene la fuente
})
# Ver las primeras filas para comprobar el resultado
print(gdf_rutas_combinadas[['ID_Ruta', 'CUADRICULA']].head())


   ID_Ruta                 CUADRICULA
0        0  28RCS53, 28RCS54, 28RCS64
1        1           28RCS43, 28RCS53
2        2           28RCS42, 28RCS43
3        3  28RCS31, 28RCS41, 28RCS42
4        4           28RCS30, 28RCS31


In [58]:
# Unir el DataFrame original de rutas con el GeoDataFrame combinado
df_rutas_con_malla = pd.merge(df_rutas_csv, gdf_rutas_combinadas[['ID_Ruta', 'CUADRICULA']], 
                               on='ID_Ruta', how='left')

# Reordenar las columnas colocando 'CUADRICULA' al inicio
columnas = ['CUADRICULA'] + [col for col in df_rutas_con_malla.columns if col != 'CUADRICULA']
df_rutas_con_malla = df_rutas_con_malla[columnas]

### Normalización de Nombres de Comunidades Autónomas y Exportación Final de Rutas con Cuadrículas Integradas

In [ ]:
# Diccionario con los nombres correctos de las comunidades autónomas
mapeo_variantes = {
    "Andalucía": ["Andalucía", "ANDALUCIA", "Andalucia"],
    "Aragón": ["ARAGON", "Aragón", "Aragon"],
    "Asturias": ["ASTURIAS"],
    "Castilla La Mancha": ["Castilla La Mancha", "CASTILLA LA MANCHA", "Castilla-La Mancha"],
    "Galicia": ["Galicia", "GALICIA"],
    "Murcia": ["Murcia", "MURCIA", "R. Murcia", "R.Murcia"],
    "Navarra": ["Navarra", "NAVARRA"],
    "Valencia": ["C. Valenciana", "Valencia", "VALENCIANA", "C. valenciana", "Valenciana", "C.valenciana", "C.Valencia"],
    "País Vasco": ["País Vasco", "PASI VASCO", "PAIS VASCO", "Pais vasco", "Pasi vasco"],
    "Castilla y León": ["Castilla y León", "Castilla y Léon", "Castilla y léon", "Castilla y leon", "Casilla y León"]
}

# Invertir el diccionario para acceder rápidamente a cada variante
nombre_estandarizado = {variante.lower(): nombre for nombre, variantes in mapeo_variantes.items() for variante in variantes}

def reemplazar_variantes(ccaa):
    if isinstance(ccaa, str):
        for variante, nombre_correcto in nombre_estandarizado.items():
            ccaa = re.sub(rf'\b{re.escape(variante)}\b', nombre_correcto, ccaa, flags=re.IGNORECASE)
    return ccaa

# Función para limpiar, separar y normalizar la columna CC_AA
def limpiar_ccaa(ccaa):
    if isinstance(ccaa, str):
        # Primero, normalizar todos los nombres incorrectos dentro de la cadena
        ccaa = reemplazar_variantes(ccaa)

        # Proteger "Castilla y León" antes de hacer el split
        ccaa = ccaa.replace("Castilla y León", "CASTILLAYLEON_TEMP")
        #ccaa = ccaa.replace("Castilla-La Mancha", "CASTILLALAMANCHA_TEMP")

        # Reemplazar " y ", "/", "-" por comas
        ccaa = re.sub(r"\s*(?:/|-| y )\s*", ", ", ccaa)

        # Separar por comas, normalizar y eliminar duplicados
        comunidades = {c.strip() for c in ccaa.split(",")}

        # Restaurar "Castilla y León"
        comunidades = {c.replace("CASTILLAYLEON_TEMP", "Castilla y León") for c in comunidades}
        #comunidades = {c.replace("Castillayleon_Temp", "Castilla y León") for c in comunidades}
        #comunidades = {c.replace("Castillalamancha_Temp", "Castilla La Mancha") for c in comunidades}

        return ", ".join(sorted(comunidades))  # Ordenar y unir con comas
    return ccaa

# Aplicar la función a la columna "CC_AA"
df_rutas_con_malla["CC_AA"] = df_rutas_con_malla["CC_AA"].apply(limpiar_ccaa)

In [ ]:
# Definir la ruta de salida
ruta_salida = "archivo_rutas_con_cuadricula.csv"

# Guardar el archivo CSV con la nueva columna
df_rutas_con_malla.to_csv(ruta_salida, index=False, encoding="utf-8-sig", decimal=",")
print(f"Archivo de rutas con cuadricula guardado en: {ruta_salida}")